In [1]:
import pandas as pd
import numpy as np
import os
from nltk.tokenize import word_tokenize
import censusgeocode as cg 

In [2]:
from IPython.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)

In [3]:
#Setting working directory
#Desktop
os.chdir("C:/Users/john/Documents/GitHub_Data/VCAP-PetInclusiveHousing_Project-Data")
#Laptop
#os.chdir('C:/Users/johnc/Documents/GitHub_Data/VCAP-PetInclusiveHousing-Data')

### Grabbing County and Census Tracts for sample of apartments

In [5]:
pet_housing_addresses = pd.read_csv("geocoded_data.csv")
#Some of the addresses did not return correct coordinates so they needed to be replaced with the correct coordinates
updated_coord = pd.read_csv("updated_coords_for_geocoding.csv")

The purpose of this script is to add the census tract to the vet clinic address. Sometimes the api times out, so I need to be able to continually append to the same list

In [16]:
pet_housing_addresses.head()
updated_coord.head()

,lon,lat,address
0,-82.001689,33.473626,"1926 howard ave, augusta, ga 30904, usa"
1,-81.999395,33.475190,"908 holden st, augusta, ga 30904, usa"
2,-82.031847,33.417546,"2902 howell rd, augusta, ga 30906, usa"
3,-84.549574,34.170196,"700 freedom blvd, canton, ga 30114, usa"
4,-84.892893,32.464177,"925 amber dr, columbus, ga 31907, usa"


,Index,lat,lon
0,287,32.455855,-84.992460
1,297,33.985287,-84.342650
2,675,33.926037,-84.567896
3,839,33.378173,-84.773683
4,2497,33.555943,-84.599163


In [77]:
pet_housing_second_batch = pet_housing_addresses.iloc[19430:, :]

In [78]:
pet_housing_second_batch.head()

,lon,lat,address
19430,-118.386838,34.185987,"6340 lankershim blvd, north hollywood, ca 9160..."
19431,-118.373165,34.165657,"5225 blakeslee ave, north hollywood, ca 91601,..."
19432,-118.392538,34.195474,"6843 1/2 radford ave, north hollywood, ca 9160..."
19433,-118.382998,34.168815,"11525 chandler blvd, north hollywood, ca 91601..."
19434,-118.380123,34.171973,"11430 burbank blvd, north hollywood, ca 91601,..."


In [21]:
len(pet_housing_addresses)

27309

In [10]:
census_tract_join_dict = {"Index" : [],
                           "full_census_tract" : []}

county_join_dict = {"Index" : [],
                          "full_county_name":[]}

In [11]:
lng = -82.001689
lat = 33.473626
result = cg.coordinates(x=lng, y=lat)

In [18]:
result["Counties"][0]

{'GEOID': '13245',
 'CENTLAT': '+33.3596223',
 'AREAWATER': '10995252',
 'STATE': '13',
 'BASENAME': 'Richmond',
 'OID': '27590389324539',
 'LSADC': '06',
 'FUNCSTAT': 'C',
 'INTPTLAT': '+33.3614863',
 'NAME': 'Richmond County',
 'OBJECTID': 50,
 'CENTLON': '-082.0735510',
 'COUNTYCC': 'H6',
 'COUNTYNS': '00358024',
 'AREALAND': '840036593',
 'INTPTLON': '-082.0749982',
 'MTFCC': 'G4020',
 'COUNTY': '245',
 'CENT': (-82.073551, 33.3596223),
 'INTPT': (-82.0749982, 33.3614863)}

In [12]:
import censusgeocode as cg 

for idx, lat, lng in zip(updated_coord["Index"], updated_coord["lat"], updated_coord["lon"]):
    try:
        result = cg.coordinates(x=lng, y=lat)
        full_census_tract = result["Census Tracts"][0]["NAME"] + "; " + result['Counties'][0]['NAME'] + "; " + result["States"][0]['BASENAME']
        full_county_name = result['Counties'][0]['NAME'] + ", " + result["States"][0]["BASENAME"]
        census_tract_join_dict["Index"].append(idx)
        census_tract_join_dict["full_census_tract"].append(full_census_tract)
        county_join_dict["Index"].append(idx)
        county_join_dict["full_county_name"].append(full_county_name)
    except ValueError:
        census_tract_join_dict["Index"].append(idx)
        census_tract_join_dict["full_census_tract"].append("Census Tract Not Found")
        county_join_dict["Index"].append(idx)
        county_join_dict["full_county_name"].append("County Not Found")
    except KeyError:
        census_tract_join_dict["Index"].append(idx)
        census_tract_join_dict["full_census_tract"].append("Census Tract Not Found")
        county_join_dict["Index"].append(idx)
        county_join_dict["full_county_name"].append("County Not Found")
        

In [13]:
census_df = pd.DataFrame(census_tract_join_dict)

In [14]:
county_df = pd.DataFrame(county_join_dict)

In [15]:
len(census_df)
len(county_df)

76

76

In [16]:
combined_df = pd.merge(census_df, county_df, on = "Index", how = "left")

In [17]:
combined_df.head(76)

,Index,full_census_tract,full_county_name
0,287,Census Tract 111; Muscogee County; Georgia,"Muscogee County, Georgia"
1,297,Census Tract 101.25; Fulton County; Georgia,"Fulton County, Georgia"
2,675,Census Tract 309.10; Cobb County; Georgia,"Cobb County, Georgia"
3,839,Census Tract 1703.07; Coweta County; Georgia,"Coweta County, Georgia"
4,2497,Census Tract 105.19; Fulton County; Georgia,"Fulton County, Georgia"
5,2763,Census Tract 505.67; Gwinnett County; Georgia,"Gwinnett County, Georgia"
6,3093,Census Tract 703.24; Henry County; Georgia,"Henry County, Georgia"
7,3122,Census Tract 703.24; Henry County; Georgia,"Henry County, Georgia"
8,3126,Census Tract 703.24; Henry County; Georgia,"Henry County, Georgia"
9,3180,Census Tract 1202.08; Paulding County; Georgia,"Paulding County, Georgia"


In [18]:
census_df.to_csv("updated_tracts.csv", index = False)
county_df.to_csv("updated_counties.csv", index = False)
combined_df.to_csv("updated_geographies.csv", index = False)

In [91]:
len(combined_df)

27725

,full_address,full_county_name
0,"1926 howard ave, augusta, ga 30904, usa","Richmond County, Georgia"
1,"908 holden st, augusta, ga 30904, usa","Richmond County, Georgia"
2,"2902 howell rd, augusta, ga 30906, usa","Richmond County, Georgia"
3,"700 freedom blvd, canton, ga 30114, usa","Cherokee County, Georgia"
4,"925 amber dr, columbus, ga 31907, usa","Muscogee County, Georgia"
...,...,...
7867,"willian mcdonald park, 5500 eastland st, fort ...","Tarrant County, Texas"
7868,"3601 deen rd, fort worth, tx 76106, usa","Tarrant County, Texas"
7869,"4850 river ranch blvd, fort worth, tx 76132, usa","Tarrant County, Texas"
7870,"3001 crockett st, fort worth, tx 76107, usa","Tarrant County, Texas"


In [40]:
other_address_tracts = pd.DataFrame(census_tract_join_dict)
other_address_tracts.tail()
len(other_address_tracts)

,full_address,full_census_tract
8897,"TIMBERLYNE ANIMAL CLINIC INC, 1703 LEGION RD #...","Census Tract 121.01, Orange County, North Caro..."
8898,"SOUTH TX VET SPECLSTS LLP, 503 E SONTERRA BLVD...","Census Tract 1918.18, Bexar County, Texas"
8899,"KEVIN A MONCE DVM, 305 ASHVILLE AVE, CARY, NC,...","Census Tract 535.09, Wake County, North Carolina"
8900,"PLANTATION ANIMAL HOSP-MTTHWS, 3036 WEDDINGTON...","Census Tract 58.51, Mecklenburg County, North ..."
8901,"ALT VET IMAGING, 7629 UNIVERSITY DR, SHREVEPOR...","Census Tract 239.04, Caddo Parish, Louisiana"


8902

In [41]:
other_addresses = other_addresses.merge(other_address_tracts, on='full_address', how='left')

In [45]:
other_addresses.to_csv("other_geo_coded_addresses.csv")

In [4]:
#Reading in Data

In [5]:
#All addresses list
all_addresses = pd.read_csv("all_geocoded_addresses.csv")
#census tract join table
#census_tract_join_table = pd.read_csv("census_tract_join_table.csv")

In [6]:
all_addresses_join_table = all_addresses[["full_address", "lat", "long"]]

In [7]:
len(all_addresses_join_table)

23637

In [32]:
#census_tract_join_dict = {"full_address" : [],
#                           "full_census_tract" : []}

In [47]:
len(census_tract_join_dict['full_address'])

16115

In [54]:
new_table = all_addresses_join_table.loc[16115:]

In [55]:
new_table.head()

,full_address,lat,long
16115,"ALVIN-FRIENDSWOOD VET CLINIC, 2407 W PARKWOOD ...",29.482395,-95.217793
16116,"EASTON COMMONS ANIMAL HOSPITAL, 15030 WEST RD,...",29.900835,-95.628488
16117,"VCA SOUTHWEST FREEWAY ANIMAL, 15575 SOUTHWEST ...",29.602112,-95.617226
16118,"VALLEY ANIMAL HOSPITAL LLC, 520 E RIDGE RD, PA...",40.320400,-76.595854
16119,"PETVET, 6763 HIGHWAY 6 S # 1800, HOUSTON, TX, ...",29.704405,-95.642775


In [56]:
import censusgeocode as cg 

for ad, lat, lng in zip(new_table["full_address"], new_table["lat"], new_table["long"]):
    try:
        result = cg.coordinates(x=lng, y=lat)
        full_census_tract = result["Census Tracts"][0]["NAME"] + ", " + result['Counties'][0]['NAME'] + ", " + result["States"][0]['NAME']
        census_tract_join_dict["full_address"].append(ad)
        census_tract_join_dict["full_census_tract"].append(full_census_tract)
    except ValueError:
        census_tract_join_dict["full_address"].append(ad)
        census_tract_join_dict["full_census_tract"].append("Not Found")

,full_address,full_census_tract
23632,"HEALING HANDS VETERINARY HOSP, 7427 N LOOP 160...","Census Tract 1820.03, Bexar County, Texas"
23633,"BROOKWOOD ANIMAL HOSP-JENNIFER, 922 DOGWOOD RD...","Census Tract 507.56, Gwinnett County, Georgia"
23634,"SHORE KAREN BVMS, 5280 PACHECO BLVD, PACHECO, ...","Census Tract 3212, Contra Costa County, Califo..."
23635,"BORGFELD ANIMAL HOSPITAL PLLC, 1304 W BORGFELD...","Census Tract 1918.04, Bexar County, Texas"
23636,"ATLANTIC ANIMAL HOSPITAL PA, 1319 MILITARY CUT...","Census Tract 106, New Hanover County, North Ca..."


In [62]:
census_tract_join_table = pd.DataFrame(census_tract_join_dict)

In [63]:
census_tract_join_table.to_csv("census_tract_join_table.csv")

In [68]:
import censusgeocode as cg 

cg.coordinates(x = -73.25840, y = 41.31197)

{'States': [{'STATENS': '01779780',
   'GEOID': '09',
   'CENTLAT': '+41.5751437',
   'AREAWATER': 1816364426,
   'STATE': '09',
   'BASENAME': 'Connecticut',
   'STUSAB': 'CT',
   'OID': '27490331955805',
   'LSADC': '00',
   'FUNCSTAT': 'A',
   'INTPTLAT': '+41.5798637',
   'DIVISION': '1',
   'NAME': 'Connecticut',
   'REGION': '1',
   'OBJECTID': 26,
   'CENTLON': '-072.7393081',
   'AREALAND': 12541750274,
   'INTPTLON': '-072.7466572',
   'MTFCC': 'G4000',
   'CENT': (-72.7393081, 41.5751437),
   'INTPT': (-72.7466572, 41.5798637)}],
 'County Subdivisions': [{'COUSUB': '48620',
   'GEOID': '0912048620',
   'CENTLAT': '+41.3379086',
   'AREAWATER': 511097,
   'STATE': '09',
   'BASENAME': 'Monroe',
   'OID': '27690697821317',
   'LSADC': '43',
   'FUNCSTAT': 'A',
   'INTPTLAT': '+41.3392358',
   'NAME': 'Monroe town',
   'OBJECTID': 23356,
   'CENTLON': '-073.2250242',
   'COUSUBCC': 'T1',
   'AREALAND': 67522444,
   'INTPTLON': '-073.2228275',
   'MTFCC': 'G4040',
   'COUSUBNS': 

In [64]:
from censusgeocode import CensusGeocode
cg = CensusGeocode(benchmark='Public_AR_Current', vintage='Census2020_Current')

In [65]:
cg.coordinates(x = -95.53871, y = 30.17833)

{'2018 State Legislative Districts - Upper': [{'POP100': 1019150,
   'GEOID': '48004',
   'CENTLAT': '+29.8772005',
   'AREAWATER': 1972985466,
   'STATE': '48',
   'BASENAME': '4',
   'OID': '212704690779008',
   'LSADC': 'LU',
   'SLDU': '004',
   'FUNCSTAT': 'N',
   'INTPTLAT': '+29.8749870',
   'NAME': 'State Senate District 4',
   'OBJECTID': 771,
   'CENTLON': '-094.7002502',
   'LSY': '2018',
   'AREALAND': 6313883172,
   'INTPTLON': '-094.7127170',
   'HU100': 396829,
   'MTFCC': 'G5210',
   'LDTYP': 'O',
   'CENT': (-94.7002502, 29.8772005),
   'INTPT': (-94.712717, 29.874987)}],
 'States': [{'STATENS': '01779801',
   'POP100': 29145505,
   'GEOID': '48',
   'CENTLAT': '+31.4277553',
   'AREAWATER': 18974670781,
   'STATE': '48',
   'BASENAME': 'Texas',
   'STUSAB': 'TX',
   'OID': '2747098452115',
   'LSADC': '00',
   'FUNCSTAT': 'A',
   'INTPTLAT': '+31.4347032',
   'DIVISION': '7',
   'NAME': 'Texas',
   'REGION': '3',
   'OBJECTID': 35,
   'CENTLON': '-099.2890828',
   'AR